In [ ]:

%reload_ext autoreload
%autoreload 2
%matplotlib inline

import math
import time
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image
from sklearn.metrics import confusion_matrix

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms




# Our libraries
from train import train_model
from model_utils import *
from predict_utils import *
from vis_utils import *
from apmeter import *
from train_valid_split import *

# some initial setup
np.set_printoptions(precision=2)
use_gpu = torch.cuda.is_available()
np.random.seed(1234)

In [ ]:
def plot_loss(trn_hist, val_hist, loss_acc):
    plt.plot(trn_hist, label='Training ' + loss_acc)
    plt.plot(val_hist, label='Validation ' + loss_acc)
    plt.legend()
    plt.xlabel('Epoch')
    plt.ylabel(loss_acc)
    plt.show()

In [ ]:
def freeze_bn(m):
    if isinstance(m, nn.BatchNorm2d):
        m.eval()

In [ ]:
use_gpu = True
cuda_available = torch.cuda.is_available()
device = torch.device("cuda" if (cuda_available and use_gpu) else "cpu")

In [ ]:
DATA_DIR = "F:/MyArticel/DTASET/MY_SPLIT_DATASETssss/stanford40/stanford40_mak_pose/"

sz = 224
batch_size = 16

In [ ]:
os.listdir(DATA_DIR)

In [ ]:
trn_dir = f'{DATA_DIR}train'
val_dir = f'{DATA_DIR}valid'

In [ ]:
os.listdir(trn_dir)

In [ ]:
trn_fnames = glob.glob(f'{trn_dir}/*/*.png')
trn_fnames[:5]

In [ ]:
# LOAD DATA
train_ds = datasets.ImageFolder(trn_dir)

In [ ]:
train_ds.classes

In [ ]:
train_ds.class_to_idx

In [ ]:
train_ds.root

In [ ]:
type(train_ds.transform)

In [ ]:

# Data augmentation and normalization for training 
train_transforms = transforms.Compose([
    transforms.Resize((sz, sz)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.1, 0.1, 0.1, 0.01),
    transforms.RandomRotation(20),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Just normalization for validation
valid_transforms = transforms.Compose([
    transforms.Resize((sz, sz)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_ds = datasets.ImageFolder(f'{DATA_DIR}train', train_transforms)
#valid_ds = datasets.ImageFolder(f'{DATA_DIR}valid', valid_transforms)

train_ds, valid_ds = train_valid_split(train_ds, 10)

train_dl = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=batch_size, shuffle=True)

train_ds_sz = len(train_ds)
valid_ds_sz = len(valid_ds)

print('Train size: {}\nValid size: {} ({:.2f})'.format(train_ds_sz, valid_ds_sz, valid_ds_sz/(train_ds_sz + valid_ds_sz)))

class_names = train_ds.mother.classes

In [ ]:
len (train_ds), len (valid_ds)

In [ ]:
# dataloader
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, 
                                       shuffle=True, num_workers=4)
valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=batch_size, 
                                       shuffle=True, num_workers=4)

In [ ]:
inputs,targets =  next(iter(train_dl))
out = torchvision.utils.make_grid(inputs, padding=3)
plt.figure(figsize=(16, 12))
imshow(out, title='random image from training data')

In [ ]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

# load pre-trained ResNet50
model = load_pretrained_resnet50(model_path=None, num_classes=7)
model.apply(freeze_bn)

# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0003, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)
# optimizer = optim.Adam(model.parameters(), lr=0.0002)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.95)

model = model.to(device)
criterion = criterion.to(device)
model

In [ ]:
# training
model, trn_loss_hist, val_loss_hist, trn_acc_hist, val_acc_hist = train_model(model, train_dl, valid_dl, criterion, optimizer, scheduler, num_epochs=20)

In [ ]:
loss_acc = 'Loss'
plot_loss(trn_loss_hist, val_loss_hist, loss_acc)

In [ ]:
loss_acc = 'Accuracy'
plot_loss(trn_acc_hist, val_acc_hist, loss_acc)

In [ ]:
for param in model.parameters():
    param.require_grad = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0003, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
model, trn_loss_hist1, val_loss_hist1, trn_acc_hist1, val_acc_hist1 = train_model(model, train_dl, valid_dl, criterion, optimizer, scheduler, num_epochs=100)

In [ ]:
trn_loss_hist.extend(trn_loss_hist1)
val_loss_hist.extend(val_loss_hist1)
trn_acc_hist.extend(trn_acc_hist1)
val_acc_hist.extend(val_acc_hist1)

In [ ]:
loss_acc = 'Loss'
plot_loss(trn_loss_hist, val_loss_hist, loss_acc)

In [ ]:
loss_acc = 'Accuracy'
plot_loss(trn_acc_hist, val_acc_hist, loss_acc)

In [ ]:
# acuracy on validation data

def evaluate_model(model, dataloader):
    model.eval()  # for batch normalization layers
    corrects = 0
    for inputs, targets in dataloader:
        inputs, targets = to_var(inputs, True), to_var(targets, True)
        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)
        corrects += (preds == targets.data).sum()
    
    print('accuracy: {:.2f}'.format(100. * corrects / len(dataloader.dataset)))

In [ ]:
# mAP
m = torch.nn.Softmax(dim=1)
def calculate_model_mAP(model, dataloader):
    mAP = APMeter()
    model.eval()  # for batch normalization layers
    corrects = 0
    for inputs, targets in dataloader:
        inputs, targets = to_var(inputs, True), to_var(targets, True)
        outputs = model(inputs)
        outputs = m(outputs)
        targets.resize_(targets.size(0), 1)
        targets = Variable(targets)
        
        one_hot = torch.cuda.FloatTensor(targets.size(0), outputs.size(1)).zero_()
        one_hot.scatter_(1, targets.data, 1)
        one_hot = Variable(one_hot)
        mAP.add(outputs,one_hot)

        #print(outputs.data)
    
    for i, value in enumerate(mAP.value()):
        print(train_ds.classes[i], ' AP: {:.2f}'.format(100. * value))
    print('mAP: {:.2f}'.format(100. * mAP.value().sum() / mAP.value().size(0)))

In [ ]:
evaluate_model(model, valid_dl)

In [ ]:
evaluate_model(model, train_dl)

In [ ]:
visualize_model(model, train_dl)

In [ ]:
visualize_model(model, valid_dl)

In [ ]:
plot_errors(model, train_dl)

In [ ]:
# confusion matrix
y_pred, y_true = predict_class(model, valid_dl)
cm = confusion_matrix(y_true, y_pred)
plot_confusion_matrix(cm, train_ds.classes, normalize=True, figsize=(12,12 ))

In [ ]:
# Look at the sizes of the images

fnames = glob.glob(f'{trn_dir}/*/*.png')
sizes = [Image.open(f).size for f in fnames]

hs, ws = list(zip(*sizes))

plt.figure(figsize=(12., 4.))
plt.hist(hs)
plt.hist(ws);

In [ ]:
visualize_model(model, valid_dl, num_images=6)

In [ ]:
plot_errors(model, valid_dl)

In [ ]:
### testing

class_names = train_ds.classes
test_dir = f'{DATA_DIR}\\test'
test_ds = datasets.ImageFolder(test_dir,valid_transforms)
len(test_ds)

In [ ]:
### dataloder

test_dl =  torch.utils.data.DataLoader(test_ds,batch_size= batch_size, num_workers=8)

In [ ]:
pred_class_names, y = predict_class_names(model, test_dl, class_names)

test_fnames= glob.glob(f'{test_dir}/*/*.png')
len(test_fnames), test_fnames [:5]

In [ ]:
test_fnames_len= len(test_fnames)
for i in range(test_fnames_len):
    test_fnames[i] = os.path.basename(test_fnames[i])
    
    len(test_fnames), test_fnames[:5]

In [ ]:
pred_result = np.stack([test_fnames, pred_class_names], axis=1)
len(pred_result), pred_result

In [ ]:
evaluate_model(model, test_dl)

In [ ]:
calculate_model_mAP(model, test_dl)

In [ ]:
plot_errors(model, test_dl)

In [ ]:
# confusion matrix
y_pred, y_true = predict_class(model, test_dl)
cm = confusion_matrix(y_true, y_pred)
plot_confusion_matrix(cm, train_ds.classes, normalize=True, figsize=(12,12 ))